In [196]:
import pandas as pd
import numpy as np
import tensorflow as tf
from data_loader import load_veterans_ds, prepare_veterans_ds
from sklearn.preprocessing import StandardScaler
from utility import InputFunction, CindexMetric, CoxPHLoss, _make_riskset
import matplotlib.pyplot as plt

import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

'''
sess = tf.compat.v1.Session()

def evaluate(tensors):
    """Evaluates Tensor or EagerTensor to Numpy `ndarray`s.
    Args:
    tensors: Object of `Tensor` or EagerTensor`s; can be `list`, `tuple`,
      `namedtuple` or combinations thereof.

    Returns:
      ndarrays: Object with same structure as `tensors` except with `Tensor` or
        `EagerTensor`s replaced by Numpy `ndarray`s.
    """
    if tf.executing_eagerly():
        return tf.contrib.framework.nest.pack_sequence_as(
            tensors,
            [t.numpy() if tf.contrib.framework.is_tensor(t) else t
             for t in tf.contrib.framework.nest.flatten(tensors)])
    return sess.run(tensors)
'''

'\nsess = tf.compat.v1.Session()\n\ndef evaluate(tensors):\n    """Evaluates Tensor or EagerTensor to Numpy `ndarray`s.\n    Args:\n    tensors: Object of `Tensor` or EagerTensor`s; can be `list`, `tuple`,\n      `namedtuple` or combinations thereof.\n\n    Returns:\n      ndarrays: Object with same structure as `tensors` except with `Tensor` or\n        `EagerTensor`s replaced by Numpy `ndarray`s.\n    """\n    if tf.executing_eagerly():\n        return tf.contrib.framework.nest.pack_sequence_as(\n            tensors,\n            [t.numpy() if tf.contrib.framework.is_tensor(t) else t\n             for t in tf.contrib.framework.nest.flatten(tensors)])\n    return sess.run(tensors)\n'

In [197]:
# Load data
X_train, X_valid, X_test, y_train, y_valid, y_test = load_veterans_ds()
t_train, t_valid, t_test, e_train, e_valid, e_test  = prepare_veterans_ds(y_train, y_valid, y_test)

iteritems is deprecated and will be removed in a future version. Use .items instead.
iteritems is deprecated and will be removed in a future version. Use .items instead.


In [198]:
# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [199]:
# Compute risk set for 5 samples
n_samples = 5
train_event_set = tf.expand_dims(e_train[:n_samples].astype(np.int32), axis=1)
train_risks_set = tf.convert_to_tensor(_make_riskset(t_train[:n_samples]))
pred_ = tf.convert_to_tensor(np.linspace(0.1, 0.9, n_samples).reshape(n_samples,1).astype(np.float32)) # random prediction

# Run loss function
loss_fn = CoxPHLoss()
loss_fn(y_true=[train_event_set, train_risks_set], y_pred=pred_)

<tf.Tensor: shape=(), dtype=float32, numpy=1.047399>

In [200]:
X_data_ = X_train[:,0] # for now, just use 1 feature TODO
Y_data_ = t_train # dont use event labels for now TODO

In [201]:
obs_stdev = tf.sqrt(tf.reduce_mean(tf.math.squared_difference(Y_data_, tf.reduce_mean(Y_data_, axis=0)), axis=0))

# Define the log probability of the bayesian regression function
def risk_posterior_log_prob(X_data_, Y_data_, alpha, beta):
  """
  Our posterior log probability, as a function of states

  Args:
    alpha_: scalar, taken from state of the HMC
    beta_: scalar, taken from state of the HMC
    sigma_: scalar, the standard deviation of , taken from state of the HMC
  Returns: 
    Scalar sum of log probabilities
  Closure over: Y_data, X_data
  """
  # TODO: Implement proper risk calculation here
  
  rv_std = tfd.Uniform(name="std", low=0., high=100.)
  rv_beta = tfd.Normal(name="beta", loc=0., scale=100.)
  rv_alpha = tfd.Normal(name="alpha", loc=0., scale=100.)

  mean = np.float32(alpha + beta * X_data_)
  rv_observed = tfd.Normal(name="obs", loc=mean, scale=1e-3) # hardcoded sigma TODO

  return (
      rv_alpha.log_prob(alpha)
      + rv_beta.log_prob(beta)
      + tf.reduce_sum(rv_observed.log_prob(Y_data_))
  )

In [202]:
'''
rv_std = tfd.Uniform(name="std", low=0., high=100.)
rv_beta = tfd.Normal(name="beta", loc=0., scale=100.)
rv_alpha = tfd.Normal(name="alpha", loc=0., scale=100.)
alpha = tf.cast(1.,dtype=tf.float32) * tf.ones([], name='init_alpha', dtype=tf.float32),
beta = tf.cast(0.01,dtype=tf.float32) * tf.ones([], name='init_beta', dtype=tf.float32),
sigma = tf.cast(obs_stdev,dtype=tf.float32) * tf.ones([], name='init_sigma', dtype=tf.float32)
mean = np.float32(alpha + beta * X_data_)
rv_observed = tfd.Normal(name="obs", loc=mean, scale=sigma)
rv_alpha.log_prob(alpha) + rv_beta.log_prob(beta) + tf.reduce_sum(rv_observed.log_prob(Y_data_)) # causes -inf TODO
'''

'\nrv_std = tfd.Uniform(name="std", low=0., high=100.)\nrv_beta = tfd.Normal(name="beta", loc=0., scale=100.)\nrv_alpha = tfd.Normal(name="alpha", loc=0., scale=100.)\nalpha = tf.cast(1.,dtype=tf.float32) * tf.ones([], name=\'init_alpha\', dtype=tf.float32),\nbeta = tf.cast(0.01,dtype=tf.float32) * tf.ones([], name=\'init_beta\', dtype=tf.float32),\nsigma = tf.cast(obs_stdev,dtype=tf.float32) * tf.ones([], name=\'init_sigma\', dtype=tf.float32)\nmean = np.float32(alpha + beta * X_data_)\nrv_observed = tfd.Normal(name="obs", loc=mean, scale=sigma)\nrv_alpha.log_prob(alpha) + rv_beta.log_prob(beta) + tf.reduce_sum(rv_observed.log_prob(Y_data_)) # causes -inf TODO\n'

In [203]:
number_of_steps = 30000
burnin = 5000

# Set the chain's start state.
initial_chain_state = [
    tf.cast(1.,dtype=tf.float32) * tf.ones([], name='init_alpha', dtype=tf.float32),
    tf.cast(0.01,dtype=tf.float32) * tf.ones([], name='init_beta', dtype=tf.float32),
]

unconstraining_bijectors = [
    tfp.bijectors.Identity(), #alpha
    tfp.bijectors.Identity(), #beta # Figure out proper bijector for beta TODO
]

unnormalized_posterior_log_prob = lambda *args: risk_posterior_log_prob(X_data_, Y_data_, *args)

In [204]:
# Initialize the step_size.
step_size = 0.5

# Defining the HMC
kernel=tfp.mcmc.TransformedTransitionKernel(
    inner_kernel=tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=unnormalized_posterior_log_prob,
        num_leapfrog_steps=2,
        step_size=step_size,
        state_gradients_are_stopped=True),        
    bijector=unconstraining_bijectors)

kernel = tfp.mcmc.SimpleStepSizeAdaptation(
    inner_kernel=kernel, num_adaptation_steps=int(burnin * 0.8))

# Sampling from the chain.
[alpha, beta, sigma], kernel_results = tfp.mcmc.sample_chain(
    num_results = number_of_steps,
    num_burnin_steps = burnin,
    current_state=initial_chain_state,
    kernel=kernel,
    name='HMC_sampling'
)

# Initialize any created variables for preconditions
init_g = tf.global_variables_initializer()

Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.


KeyboardInterrupt: 